# 네이버 뉴스 크롤 만들기
date: 2024-01-17
written by: [Jehwan Kim](github.com/kreimben)

In [1]:
# !pip install -r requirements.txt --upgrade

In [2]:
import traceback
from datetime import datetime, timedelta
from time import sleep

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
end_date = datetime.now()
start_date = end_date - timedelta(days=2)  # 최근 3일간의 데이터를 위한 설정

end_date = end_date.strftime("%Y%m%d")
start_date = start_date.strftime("%Y%m%d")

query = '데이터분석'
url = f"https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={start_date}&de={end_date}&docid=&nso=so:r,p:from{start_date}to{end_date},a:all&mynews=0&refresh_start=0&related=0"
max_page = 50  # 크롤링을 원하는 최대 페이지 수 지정

start_date, end_date

('20240115', '20240117')

In [4]:
# 각 기사들의 데이터를 종류별로 나눠담을 리스트를 생성합니다. (추후 DataFrame으로 모을 예정)
titles = []
dates = []
articles = []
article_urls = []
press_companies = []
categories = []
category_kind = {
    '정치': 100, '경제': 101, '사회': 102, '생활/문화': 103, "세계": 104, "IT/과학": 105, '연예': 106, '스포츠': 107
}

# 지정한 기간 내 원하는 페이지 수만큼의 기사를 크롤링합니다.
current_call = 1
last_call = (max_page - 1) * 10 + 1  # max_page이 5일 경우 41에 해당 (1페이지는 url에 포함되어 있으므로 1을 빼줌)

# For error rate calulation
errors = []

# For exclude duplicated articles
visit = []

In [5]:
while current_call <= last_call:

    print('\n{}번째 기사글부터 크롤링을 시작합니다.'.format(current_call))

    url = "https://search.naver.com/search.naver?where=news&query=" + query \
          + "&nso=so%3Ar%2Cp%3Afrom" + start_date \
          + "to" + end_date \
          + "%2Ca%3A&start=" + str(current_call)

    web = requests.get(url).content
    source = BeautifulSoup(web, 'html.parser')

    urls_list = []
    for urls in source.find_all('a', {'class': "info"}):
        if urls["href"].startswith("https://n.news.naver.com"):
            urls_list.append(urls["href"])

    for url in urls_list:
        # 중복 기사 제거
        if url in visit:
            continue
        else:
            visit.append(url)
            
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
            web_news = requests.get(url, headers=headers).content
            source_news = BeautifulSoup(web_news, 'html.parser')

            if title := source_news.find('h2', {'class': 'media_end_head_headline'}):
                title = title.get_text()
            else:
                title = source_news.find('h2', {'class': 'end_tit'}).get_text()
            print('Processing article : {}'.format(title))

            date = source_news.find('span', {'class': 'media_end_head_info_datestamp_time'}).get_text()

            article = source_news.find('article', {'id': 'dic_area'}).get_text()
            article = article.replace("\n", "")
            article = article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
            article = article.replace("동영상 뉴스       ", "")
            article = article.replace("동영상 뉴스", "")
            article = article.strip()

            press_company = source_news.find('em', {'class': 'media_end_linked_more_point'}).get_text()

            titles.append(title)
            dates.append(date)
            articles.append(article)
            press_companies.append(press_company)
            article_urls.append(url)

            for k, v in category_kind.items():
                find = f'sid={v}'
                target = url
                if find in target:
                    categories.append(k)
                    break
            else:
                categories.append('N/A')
        except Exception as e:
            print(f'*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : {url}')
            print(f'에러 내용 : {e}')
            print(traceback.format_exc())
            errors.append(url)

    # 대량의 데이터를 대상으로 크롤링을 할 때에는 요청 사이에 쉬어주는 타이밍을 넣는 것이 좋습니다.
    sleep(1)
    current_call += 10


1번째 기사글부터 크롤링을 시작합니다.
Processing article : 이차전지 10년 투자로 7200% 벌었다…신한證, 빅데이터 공개
Processing article : 마약류 오남용 통합감시 시스템 구축…"불법 사전차단"
Processing article : K-뷰티 견인하는 화장품 천연소재, 특허 빅데이터 분석해 찾는다
Processing article : '체납징수' 팔걷은 정선군…"전담반에 빅데이터 분석도"
Processing article : [가상자산 2024] 주기영 크립토퀀트 대표 “블록체인 데이터 분석으로 투명한 투자 정보 발굴”
Processing article : 기업은행, 데이터본부 신설하고 신탁 등 비이자부문 강화

11번째 기사글부터 크롤링을 시작합니다.
Processing article : NIA, 빅데이터 분석 지원 수요 기업 모집
Processing article : IBK기업은행, 데이터본부 신설…박일규·오은선 부행장 선임
Processing article : "데이터 사용량 분석해 최적 구간 추천"…토스모바일, '맞춤 요금제' 출시
Processing article : AI 데이터센터 관리솔루션 CES 선보인 SK브로드밴드
Processing article : 소방청장 "현장대원 생체데이터 변화, 실시간으로 수집한다"(종합)
Processing article : "청신경 종양환자, '이 기법' MRI 분석땐 청각상태 예측“
Processing article : NIA, '2024년 빅데이터 플랫폼 기반 분석서비스' 지원 수요기업 모집
Processing article : 데이터스트림즈, 가트너 매직쿼드런트 또 등재

21번째 기사글부터 크롤링을 시작합니다.
Processing article : 글로벌 분석기관들 “기업생존 ‘생성형 AI’에 달렸다”
Processing article : 지하철·건물 내, 데이터 잘 터지나?…SKT, 3차원 분석 기술 적용
Processing article : AI번역 파파고, 구글 번

In [6]:
# Dataset Length Check
print(f'Titles: {len(titles)}')
print(f'Dates: {len(dates)}')
print(f'Articles: {len(articles)}')
print(f'Article URLs: {len(article_urls)}')
print(f'Press Companies: {len(press_companies)}')
if not (len(titles) == len(dates) == len(articles) == len(article_urls) == len(press_companies)):
    raise ValueError('Dataset Length is not equal')

print(f'{len(errors)} errors occured')

Titles: 119
Dates: 119
Articles: 119
Article URLs: 119
Press Companies: 119
4 errors occured


In [7]:
# 각 데이터 종류별 list에 담아둔 전체 데이터를 DataFrame에 모으고 엑셀 파일로 저장합니다.
# 파일명을 result_연도월일_시분.csv 로 지정합니다.
article_df = pd.DataFrame({
    'title': titles,
    'date': dates,
    'document': articles,
    'link': article_urls,
    'press': press_companies,
    'category': categories
})

article_df.to_csv(f'result_from_{start_date}_end_{end_date}.csv', index=False, encoding='utf-8')

In [8]:
article_df.sort_values(by='date', ascending=True)

,title,date,document,link,press,category
111,"토스모바일, 업계 최초 '사용량 맞춤 요금제' 출시",2024.01.15. 오전 10:16,통신요금 적게 쓰면 다음달 요금제 하향 추천…최대 3만4천800원토스모바일[토스모바...,https://n.news.naver.com/mnews/article/001/001...,연합뉴스,IT/과학
13,"데이터스트림즈, 가트너 매직쿼드런트 또 등재",2024.01.15. 오전 10:23,작년 12월 '2023년 데이터통합 툴 분야'서 아너러블 벤더로 이름 올려데이터스트...,https://n.news.naver.com/mnews/article/092/000...,지디넷코리아,IT/과학
114,"""AI반도체 1등""…판교 R&D 허브 조성·연구인력 2배 이상 확충",2024.01.15. 오전 10:35,"판교·수원·평택, R&D·교육 거점…""저전력으로 엔비디아 능가""4년간 연구자 2천 ...",https://n.news.naver.com/mnews/article/001/001...,연합뉴스,IT/과학
47,"제이제이앤컴퍼니스, 낙동강 어도(Fishway) 수중 생태환경 분석 시스템 구축 완료",2024.01.15. 오전 10:38,해양엔지니어링 기업인 제이제이앤컴퍼니스(대표 전정호)가 인공지능을 활용한 ‘수중 생...,https://n.news.naver.com/mnews/article/015/000...,한국경제,경제
54,"샌즈랩 컨소시엄, KISA 사이버보안 AI 데이터셋 구축 완료",2024.01.15. 오전 11:29,위협 인텔리전스 데이터셋의 구성 / 사진제공=샌즈랩사이버 위협 인텔리전스 전문기업 ...,https://n.news.naver.com/mnews/article/008/000...,머니투데이,IT/과학
...,...,...,...,...,...,...
60,"""올해는 글로벌 R&D 혁신 원년""…1.8조 투입하고 특위도 신설",2024.01.17. 오후 12:02,시스템 혁신·국내 연구자 역량 강화·생태계 조성 등 3개 방향 추진글로벌 R&D 연...,https://n.news.naver.com/mnews/article/003/001...,뉴시스,IT/과학
83,"순천시, 이웃이 이웃을 돌보는 생활밀착 돌봄체계 구축",2024.01.17. 오후 1:22,"AI 안부전화·초기상담, ICT로 돌봄 일상화…배려돌봄 넘어 정서돌봄·관계돌봄 확장...",https://n.news.naver.com/mnews/article/008/000...,머니투데이,사회
71,"순천시, ‘생활밀착 돌봄체계’ 구축",2024.01.17. 오후 1:56,순천시 야경 모습 =순천시 제공[스포츠서울/조광태기자] 전남 순천시(노관규 순천시장...,https://n.news.naver.com/mnews/article/468/000...,스포츠서울,사회
98,"과기정통부, 글로벌 R&D 1조 8167억 투입 총력전",2024.01.17. 오후 1:57,"- 글로벌 R&D 특별위원회 신설, 가이드라인 마련 등 본격 추진정부출연연구기관 연...",https://n.news.naver.com/mnews/article/016/000...,헤럴드경제,IT/과학


In [9]:
article_df.category.isna().sum()
# 카테고리 분류가 제대로 되지 않은 뉴스 기사는 없다.

0

In [10]:
article_df.count().sum()
# 792개의 기사가 있다.

714